In [1]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from dotenv import load_dotenv
import json

In [2]:
# Cargar variables de entorno
load_dotenv()

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(
    temperature=0,  # Configuramos temperatura baja para respuestas consistentes
    model="gpt-4o",
)

In [13]:
# PASO 1: Extracción de datos
extraction_template = """
Analiza el siguiente email de solicitud de devolución y extrae los datos relevantes.

Email:
```
{email_content}
```

Extrae los siguientes datos en formato JSON:
- numero_pedido: El número o código de referencia del pedido.
- cliente_nombre: Nombre completo del cliente.
- cliente_departamento: Departamento o área a la que pertenece el cliente, si se menciona.
- producto: Descripción del producto afectado, incluyendo modelo si se menciona.
- motivo_solicitud: Motivo principal por el que solicita la devolución o reemplazo.
- detalles_problema: Explicación detallada del problema reportado.
- datos_contacto: Información de contacto proporcionada en el email.

{format_instructions}
"""

# Definir el esquema para la extracción de datos
extraction_schema = [
    ResponseSchema(
        name="numero_pedido", description="Número o código de referencia del pedido"
    ),
    ResponseSchema(name="cliente_nombre", description="Nombre completo del cliente"),
    ResponseSchema(
        name="cliente_departamento",
        description="Departamento o área a la que pertenece el cliente",
    ),
    ResponseSchema(name="producto", description="Descripción del producto afectado"),
    ResponseSchema(
        name="motivo_solicitud", description="Motivo principal de la solicitud"
    ),
    ResponseSchema(
        name="detalles_problema", description="Explicación detallada del problema"
    ),
    ResponseSchema(
        name="datos_contacto", description="Información de contacto proporcionada"
    ),
]

extraction_parser = StructuredOutputParser.from_response_schemas(extraction_schema)
format_instructions = extraction_parser.get_format_instructions()

extraction_prompt = ChatPromptTemplate.from_template(extraction_template)
extraction_chain = LLMChain(
    llm=llm, prompt=extraction_prompt, output_key="extracted_data", verbose=True
)

In [9]:
# PASO 2: Análisis de elegibilidad
eligibility_template = """
Basándote en los siguientes datos extraídos de un email de solicitud de devolución:

```
{extracted_data}
```

Determina si esta solicitud debe ser ACEPTADA o RECHAZADA según estos criterios:

MOTIVOS PARA ACEPTAR:
- Defecto de fabricación confirmado: El producto presenta fallos internos o de funcionamiento no atribuibles al transporte ni a un mal uso.
- Error en el suministro: Se han entregado componentes incorrectos en cuanto a modelo, cantidad o especificación respecto al pedido original.
- Producto incompleto o con elementos faltantes de fábrica: Falta documentación técnica, piezas necesarias o embalaje original desde el origen.

MOTIVOS PARA RECHAZAR:
- Daños ocasionados durante el transporte: Si el transporte no estaba asegurado o contratado directamente por la empresa, no se asume responsabilidad por los daños ocurridos durante el envío.
- Manipulación indebida por parte del cliente: Instalación incorrecta, modificaciones o uso inapropiado del componente.
- Superación del plazo máximo para devoluciones: La solicitud se presenta fuera del periodo establecido por la política de devoluciones (por ejemplo, 14 días naturales).

Proporciona tu decisión (ACEPTAR/RECHAZAR) y una justificación detallada basada en los criterios establecidos.

{format_instructions}
"""

# Definir esquema para análisis de eligibilidad
eligibility_schema = [
    ResponseSchema(name="decision", description="Decisión final: ACEPTAR o RECHAZAR"),
    ResponseSchema(
        name="justificacion",
        description="Explicación detallada que justifica la decisión",
    ),
    ResponseSchema(
        name="motivo_principal",
        description="Motivo específico de la política que se aplica en este caso",
    ),
    ResponseSchema(
        name="recomendaciones",
        description="Recomendaciones adicionales o alternativas para el cliente",
    ),
]

eligibility_parser = StructuredOutputParser.from_response_schemas(eligibility_schema)
eligibility_format_instructions = eligibility_parser.get_format_instructions()

eligibility_prompt = ChatPromptTemplate.from_template(eligibility_template)
eligibility_chain = LLMChain(
    llm=llm, prompt=eligibility_prompt, output_key="eligibility_result", verbose=True
)

In [10]:
# PASO 3: Generación de respuesta
response_template = """
Genera una respuesta formal para un email de solicitud de devolución con los siguientes datos:

DATOS DEL CLIENTE:
```
{extracted_data}
```

ANÁLISIS DE ELEGIBILIDAD:
```
{eligibility_result}
```

La respuesta debe:
1. Ser profesional y cortés
2. Incluir un saludo personalizado dirigido al cliente por su nombre
3. Referenciar claramente el número de pedido y producto
4. Explicar la decisión tomada de manera clara y empática
5. Incluir la justificación basada en las políticas de la empresa
6. Detallar los siguientes pasos según corresponda:
   - Si es ACEPTADA: proceso de reemplazo y plazos estimados
   - Si es RECHAZADA: alternativas disponibles para el cliente
7. Incluir una disculpa apropiada por las molestias
8. Ofrecer asistencia adicional
9. Incluir una despedida formal con datos de contacto

Firma como "Departamento de Atención al Cliente - Componentes Intergalácticos Industriales S.A."
"""

response_prompt = ChatPromptTemplate.from_template(response_template)
response_chain = LLMChain(llm=llm, prompt=response_prompt, verbose=True)

In [11]:
# Función para procesar un email completo
def procesar_solicitud_devolucion(email_content):
    # Paso 1: Extraer datos del email
    extraction_messages = extraction_prompt.format_messages(
        email_content=email_content, format_instructions=format_instructions
    )
    extraction_response = llm.invoke(extraction_messages)
    extracted_data = extraction_parser.parse(extraction_response.content)
    print("\n--- DATOS EXTRAÍDOS ---")
    print(json.dumps(extracted_data, indent=2, ensure_ascii=False))

    # Paso 2: Analizar elegibilidad
    eligibility_messages = eligibility_prompt.format_messages(
        extracted_data=json.dumps(extracted_data, indent=2, ensure_ascii=False),
        format_instructions=eligibility_format_instructions,
    )
    eligibility_response = llm.invoke(eligibility_messages)
    eligibility_result = eligibility_parser.parse(eligibility_response.content)
    print("\n--- ANÁLISIS DE ELEGIBILIDAD ---")
    print(json.dumps(eligibility_result, indent=2, ensure_ascii=False))

    # Paso 3: Generar respuesta
    response_messages = response_prompt.format_messages(
        extracted_data=json.dumps(extracted_data, indent=2, ensure_ascii=False),
        eligibility_result=json.dumps(eligibility_result, indent=2, ensure_ascii=False),
    )
    response_final = llm.invoke(response_messages)
    print("\n--- RESPUESTA GENERADA ---")
    print(response_final.content)

    return {
        "datos_extraidos": extracted_data,
        "analisis_eligibilidad": eligibility_result,
        "respuesta_email": response_final.content,
    }

In [14]:
# Ejemplo de uso con el caso de Darth Márquez
if __name__ == "__main__":
    email_ejemplo = """
    Asunto: Solicitud de reemplazo por daños en transporte – Pedido #D347-STELLA
    
    Estimado equipo de Componentes Intergalácticos Industriales S.A.,
    
    Me pongo en contacto con ustedes como cliente reciente para comunicar una
    incidencia relacionada con el pedido #D347-STELLA, correspondiente a un lote de
    condensadores de fluzo modelo FX-88, destinados a un proyecto estratégico de gran
    envergadura: la construcción de la Estrella de la Muerte.
    
    Lamentablemente, al recibir el envío, observamos que varios de los condensadores
    presentaban daños visibles y no funcionales. Tras revisar el estado del embalaje y
    consultar con el piloto de carga, todo indica que la mercancía sufrió una caída
    durante el transporte interestelar.
    
    Dado que estos componentes son críticos para la activación del núcleo central del
    sistema de rayos destructores, les solicitamos con carácter urgente el reemplazo
    inmediato de las unidades defectuosas, así como una revisión de los protocolos de
    embalaje y transporte para evitar que algo así vuelva a ocurrir.
    
    Adjunto imágenes del estado de los condensadores y el albarán de entrega sellado
    por nuestro droide de recepción.
    
    Agradezco de antemano su pronta atención a este asunto. Quedamos a la espera de
    su respuesta para coordinar el reemplazo.
    
    Atentamente,
    Darth Márquez
    Departamento de Ingeniería Imperial
    Sector de Proyectos Especiales
    Contacto: dmarquez@imperiumgalactic.net
    Holofono: +34 9X9 123 456
    """

    resultado = procesar_solicitud_devolucion(email_ejemplo)


--- DATOS EXTRAÍDOS ---
{
  "numero_pedido": "D347-STELLA",
  "cliente_nombre": "Darth Márquez",
  "cliente_departamento": "Departamento de Ingeniería Imperial, Sector de Proyectos Especiales",
  "producto": "condensadores de fluzo modelo FX-88",
  "motivo_solicitud": "Reemplazo por daños en transporte",
  "detalles_problema": "Varios condensadores presentaban daños visibles y no funcionales debido a una caída durante el transporte interestelar.",
  "datos_contacto": "dmarquez@imperiumgalactic.net, Holofono: +34 9X9 123 456"
}

--- ANÁLISIS DE ELEGIBILIDAD ---
{
  "decision": "RECHAZAR",
  "justificacion": "La solicitud de devolución se basa en daños ocasionados durante el transporte, lo cual no es un motivo aceptable para la devolución según la política establecida. La responsabilidad de los daños durante el transporte recae en el cliente si el transporte no estaba asegurado o contratado directamente por la empresa.",
  "motivo_principal": "Daños ocasionados durante el transporte",
 